IPFS Infrastructure Usage Logs (3.22 GB)</br>
cid: bafybeiftyvcar3vh7zua3xakxkb2h5ppo4giu5f3rkpsqgcfh7n7axxnsa

# Read log data

In [ ]:
import pandas as pd
import re
pd.set_option('max_columns', None)
pd.options.display.max_colwidth = 100

In [ ]:
# read log data
path = 'nginx-01-02-2021-bank2-sv15_encrypted.log'
file = open(path)
lines = file.readlines()

In [ ]:
# split by '"'
s = pd.Series(lines)
df = s.str.split('"', expand=True)

# remove rows with missing value
df['check1'] = df.apply(lambda x: len(x[1].split(' ')), axis=1)
df['check2'] = df.apply(lambda x: len(x[2].split(' ')), axis=1)
df = df[(df['check1'] == 3) & (df['check2'] == 9)].drop(['check1', 'check2'], axis=1)

In [ ]:
# create dataframe
df[['encryptedIP','timestamp']] = df[0].str.split(' ', expand=True).drop([1,2,4], axis=1)
df['timestamp'] = df['timestamp'].str.strip('[]')

df[['method','path','version']] = df[1].str.split(' ', expand=True)

df[['response', 'bytes_returned', 'request_length', 'request_time', 'upstream_response_time', 'upstream_header_time', 'cache']] = df[2].str.split(' ', expand=True).drop([0,8], axis=1)
df['bytes_returned'] = df['bytes_returned'].astype(int)

df[['referrer','agent']] = df[[3,5]]

df[['server_name','http_host','http_schema']] = df[6].str.split(' ', expand=True).drop([0], axis=1)
df['http_schema'] = df['http_schema'].str.strip('\n')

df = df.drop([0,1,2,3,4,5,6], axis=1)

df.shape

In [ ]:
# keep only GET requests
df = df[df['method']=='GET']
# filter valid agents
df = df[df['agent']!='yes']
df['check'] = df.apply(lambda x: re.search("^[a-zA-Z]", str(x['agent']))==None, axis=1)
df = df[df['check']==False].drop(['check'], axis=1)
# exclude path '/'
df = df[df['path']!='/']

df.shape

In [ ]:
# remove data from 01-03
df['datetime'] = pd.to_datetime(df['timestamp'])
df = df[df['datetime'].dt.day==2]
df = df.drop(['datetime'], axis=1)

# Extract CID

In [ ]:
df['cid'] = ''
# extract CID from path
df['http_host'] = df['http_host'].str.strip(':443')
df.loc[df['path'].str.startswith('/ipfs'), 'cid'] = df['path'].str[6:]
df.loc[df['http_host'].str.endswith('.ipfs.dweb.link'), 'cid'] = df['http_host'].str[:-15]
df['cid'] = df.apply(lambda x: x['cid'].split('/')[0], axis=1)
df.head()

In [ ]:
# remove nan
df = df[df['cid'].isna()!=True]
# filter valid cid
df['check'] = df.apply(lambda x: bool(re.match("^[A-Za-z0-9]*$", str(x['cid']))) and len(str(x['cid']))>20, axis=1)
df = df[df['check']==True].drop(['check'], axis=1)
df.shape

In [ ]:
df = df[['timestamp', 'bytes_returned', 'agent', 'cid']]
df.head()

In [ ]:
df.to_csv('data.csv') # 1.51 GB

# Group by user

In [ ]:
df = pd.read_csv('data.csv', index_col=0)
df.shape

In [ ]:
df_groupby_user = df[['bytes_returned','agent']].groupby('agent').agg(['sum','count'])
df_groupby_user.columns = df_groupby_user.columns.get_level_values(0) + '_' + df_groupby_user.columns.get_level_values(1)
df_groupby_user = df_groupby_user.reset_index()
df_groupby_user = df_groupby_user.rename(columns={
                          "bytes_returned_sum": "request_sum",
                          "bytes_returned_count": "request_count",
                         })
df_groupby_user.shape

In [ ]:
df_groupby_user.head()

In [ ]:
df_groupby_user.to_csv('data_groupby_user.csv') # 4 MB